In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.6 MB/s eta 0:00:00


In [2]:
from Bio.Seq import Seq
import random

In [3]:
# Utility class with static methods
class SequenceUtils:
    @staticmethod
    def transcribe_dna_to_rna(dna):
        return dna.transcribe()

    @staticmethod
    def translate_rna_to_protein(rna):
        return rna.translate()

In [4]:
# Singleton Storage Class
class SequenceStorage:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(SequenceStorage, cls).__new__(cls)
            cls._instance.data = {}
        return cls._instance

    def save(self, name, seq):
        self.data[name] = seq

    def read(self, name):
        return self.data.get(name)


In [5]:
# Sequence Generator Abstract Base Class
class SequenceGenerator:
    def create_sequence(self, length):
        raise NotImplementedError("Subclasses must implement create_sequence")


In [6]:
class DNASequenceGenerator(SequenceGenerator):
    alphabet = ['A', 'C', 'G', 'T']

    def create_sequence(self, length):
        return ''.join(random.choices(DNASequenceGenerator.alphabet, k=length))

class ProteinSequenceGenerator(SequenceGenerator):
    alphabet = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I',
                'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

    def create_sequence(self, length):
        return ''.join(random.choices(ProteinSequenceGenerator.alphabet, k=length))


In [7]:
class SequenceFactory:
    def __init__(self, sequence_type):
        self.sequence_type = sequence_type.lower()

    def create_sequence(self, length):
        if self.sequence_type == 'dna':
            return DNASequenceGenerator().create_sequence(length)
        elif self.sequence_type == 'protein':
            return ProteinSequenceGenerator().create_sequence(length)
        else:
            raise ValueError(f"Unknown sequence type: {self.sequence_type}")


In [8]:
# Classes for handling DNA and Protein objects
class DNASequence:
    def __init__(self, sequence):
        self.sequence = sequence

    def to_protein(self):
        rna = SequenceUtils.transcribe_dna_to_rna(Seq(self.sequence))
        return SequenceUtils.translate_rna_to_protein(rna)

class ProteinSequence:
    def __init__(self, sequence):
        self.sequence = sequence

    def get_sequence(self):
        return self.sequence

In [9]:
# Start with a known DNA sequence
example_dna = 'GTGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG'
dna_seq = Seq(example_dna)

rna_seq = SequenceUtils.transcribe_dna_to_rna(dna_seq)
protein_seq = SequenceUtils.translate_rna_to_protein(rna_seq)

print("Original DNA:   ", dna_seq)
print("Transcribed RNA:", rna_seq)
print("Protein:        ", protein_seq)

# Store values using singleton
storage = SequenceStorage()
storage.save("my_dna", dna_seq)
storage.save("my_protein", protein_seq)

print("Retrieved DNA:     ", storage.read("my_dna"))
print("Retrieved Protein: ", storage.read("my_protein"))

# Generate random DNA and Protein sequences
dna_factory = SequenceFactory('dna')
random_dna = dna_factory.create_sequence(36)
print("Random DNA:        ", random_dna)
storage.save("rand_dna", random_dna)

protein_factory = SequenceFactory('protein')
random_protein = protein_factory.create_sequence(20)
print("Random Protein:    ", random_protein)
storage.save("rand_protein", random_protein)

# Use DNA object class to convert and access
dna_obj = DNASequence(random_dna)
print("Translated Random DNA to Protein:", dna_obj.to_protein())

# Use Protein object class to retrieve
protein_obj = ProteinSequence(random_protein)
print("Stored Protein (obj):", protein_obj.get_sequence())


Original DNA:    GTGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG
Transcribed RNA: GUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
Protein:         VAIVMGR*KGAR*
Retrieved DNA:      GTGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG
Retrieved Protein:  VAIVMGR*KGAR*
Random DNA:         ATAAGTTTTGGCTTCCAACATGGCTTTCATAGGCGT
Random Protein:     RMMTVNSMEPYCWYIDWKLK
Translated Random DNA to Protein: ISFGFQHGFHRR
Stored Protein (obj): RMMTVNSMEPYCWYIDWKLK
